# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
## Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,agadez,19.7500,10.2500,29.21,8,0,5.26,NE,1717982882
1,1,grytviken,-54.2811,-36.5092,-7.11,77,22,5.18,GS,1717982882
2,2,puerto natales,-51.7236,-72.4875,1.41,99,100,2.33,CL,1717982882
3,3,blackmans bay,-43.0167,147.3167,10.20,62,5,1.79,AU,1717982883
4,4,west island,-12.1568,96.8225,27.99,83,40,3.60,CC,1717982827


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 400,
    size = "Humidity",
    scale = 0.8,
    color = "City"       
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal_df = city_data_df[(city_data_df["Humidity"] > 30) & (city_data_df["Humidity"] < 60)]

# Drop any rows with null values
city_ideal_df = city_ideal_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,agadez,19.7500,10.2500,29.21,8,0,5.26,NE,1717982882
1,1,grytviken,-54.2811,-36.5092,-7.11,77,22,5.18,GS,1717982882
2,2,puerto natales,-51.7236,-72.4875,1.41,99,100,2.33,CL,1717982882
3,3,blackmans bay,-43.0167,147.3167,10.20,62,5,1.79,AU,1717982883
4,4,west island,-12.1568,96.8225,27.99,83,40,3.60,CC,1717982827
...,...,...,...,...,...,...,...,...,...,...
546,546,kralendijk,12.1500,-68.2667,28.99,79,75,7.20,BQ,1717982691
547,547,canico,32.6333,-16.8500,20.12,84,75,2.06,PT,1717982956
548,548,pitea,65.3172,21.4794,10.05,93,97,2.24,SE,1717982956
549,549,zhuanghe,39.7008,122.9911,21.68,75,82,4.54,CN,1717982957


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,qamdo,CN,31.1667,97.2333,49,
29,bilibino,RU,68.0546,166.4372,39,
43,turkmenbasy,TM,40.0222,52.9552,34,
81,nova olimpia,BR,-14.7972,-57.2881,53,
82,ath thawrah,SY,35.8469,38.5443,51,
...,...,...,...,...,...,...
521,fort saskatchewan,CA,53.7168,-113.2187,54,
522,formosa do rio preto,BR,-11.0483,-45.1931,50,
525,muravlenko,RU,63.7898,74.5230,54,
533,tommot,RU,58.9564,126.2925,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": "",
    "bias": ""
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat}"
    params["bias"] = f"proximity:{lon},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url + "categories=accommodation.hotel" + "&bias=" + params["bias"] +"&format=json&apiKey=" + geoapify_key)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
qamdo - nearest hotel: 银杏商务宾馆
bilibino - nearest hotel: No hotel found
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
nova olimpia - nearest hotel: Olímpia Hotel
ath thawrah - nearest hotel: Al Sari
corinto - nearest hotel: Apollon
yanchep - nearest hotel: Yanchep Inn
plastun - nearest hotel: Россияночка
alice springs - nearest hotel: Aurora Alice Springs
vazante - nearest hotel: Kabana
greencastle - nearest hotel: Inn at DePauw
yellowknife - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
hamilton - nearest hotel: North Vista Manor
yeppoon - nearest hotel: Blossom's on Seaspray
bulgan - nearest hotel: Bulgan Hotel
nadi - nearest hotel: President Hotel
ruidoso - nearest hotel: Hotel Ruidoso
port lincoln - nearest hotel: Boston Hotel
kalaleh - nearest hotel: هتل تریا
anadyr - nearest hotel: Гостевой дом
east london - nearest hotel: No hotel found
bafoulabe - nearest hotel: Radio Rurale
santa ana de yacuma - nearest hotel: Hotel Yuliana
ust-nera - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
7,qamdo,CN,31.1667,97.2333,49,银杏商务宾馆
29,bilibino,RU,68.0546,166.4372,39,No hotel found
43,turkmenbasy,TM,40.0222,52.9552,34,Türkmenbaşy Oteli
81,nova olimpia,BR,-14.7972,-57.2881,53,Olímpia Hotel
82,ath thawrah,SY,35.8469,38.5443,51,Al Sari
...,...,...,...,...,...,...
521,fort saskatchewan,CA,53.7168,-113.2187,54,No hotel found
522,formosa do rio preto,BR,-11.0483,-45.1931,50,Pousada Universo - Formosa do Rio Preto BA
525,muravlenko,RU,63.7898,74.5230,54,Е-Отель
533,tommot,RU,58.9564,126.2925,54,Томмот


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 400,
    scale = 1,
    color = "City",
    hover_cols=["Lng", "Lat", "City", "Country", "Hotel Name"]
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)